In [22]:
 ! python -m spacy download sv_core_news_md

     ---------------------------------------- 0.0/42.8 MB ? eta -:--:--
     --------------------------------------- 0.0/42.8 MB 640.0 kB/s eta 0:01:07
     --------------------------------------- 0.1/42.8 MB 525.1 kB/s eta 0:01:22
     ---------------------------------------- 0.2/42.8 MB 1.6 MB/s eta 0:00:27
      --------------------------------------- 0.8/42.8 MB 4.8 MB/s eta 0:00:09
     - -------------------------------------- 1.5/42.8 MB 6.9 MB/s eta 0:00:07
     -- ------------------------------------- 2.3/42.8 MB 8.0 MB/s eta 0:00:06
     -- ------------------------------------- 3.0/42.8 MB 9.5 MB/s eta 0:00:05
     --- ------------------------------------ 3.7/42.8 MB 9.9 MB/s eta 0:00:04
     ---- ----------------------------------- 4.4/42.8 MB 10.8 MB/s eta 0:00:04
     ---- ----------------------------------- 5.2/42.8 MB 11.3 MB/s eta 0:00:04
     ----- ---------------------------------- 5.9/42.8 MB 11.4 MB/s eta 0:00:04
     ------ --------------------------------- 6.6/42.8

In [10]:
#Import libraries
import os
import pinecone
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI
from dotenv import load_dotenv
from langchain_experimental.data_anonymizer import PresidioAnonymizer
from langchain_experimental.data_anonymizer import PresidioReversibleAnonymizer


In [29]:
%pip install --upgrade --quiet langdetect

Note: you may need to restart the kernel to use updated packages.


In [27]:
import spacy
from langdetect import detect
from langchain.schema import runnable

# Numbers starting with the country code +46, followed by 8 to 10 digits.
# Numbers in a group format that might be separated by spaces or dashes.
swedish_phone_regex = r'\+?46\d{8,10}|\d{2,3}[-\s]?\d{2,3}[-\s]?\d{2,3}[-\s]?\d{2,4}'

email_regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b'


# Load the Swedish model from spaCy
nlp_sv = spacy.load("sv_core_news_md")

def anonymize_with_spacy(text: str, language: str) -> str:
    text = re.sub(email_regex, '[ANONYMIZED]', text)
    if language == 'sv':
        # Anonymize Swedish phone numbers 
        text = re.sub(swedish_phone_regex, '[ANONYMIZED]', text)

        doc = nlp_sv(text)
        anonymized_text = ""
        last_end = 0
        for ent in doc.ents:
            # Add text before the entity
            anonymized_text += text[last_end:ent.start_char]
            # Add the anonymized form of the entity
            anonymized_text += '[ANONYMIZED]'
            last_end = ent.end_char
        # Add the remaining text after the last entity
        anonymized_text += text[last_end:]
        return anonymized_text

    return text  # Return the original text if not Swedish

def detect_language_and_anonymize(text: str) -> dict:
    language = detect(text)
    anonymized_text = anonymize_with_spacy(text, language)
    print(f"Detected language: {language}")
    print(f"Anonymized text: {anonymized_text}")
    return {"text": anonymized_text, "language": language}

chain = runnable.RunnableLambda(detect_language_and_anonymize)

# Test the setup
test_text = "hej, jag heter Felix och du kan nå mig på 076-1234567 eller felix.2000@gmail.com och jag bor i Sollentuna 19164 på Blåklockevägen 24"
result = chain.invoke(test_text)

Detected language: sv
Anonymized text: hej, jag heter [ANONYMIZED] och du kan nå mig på [ANONYMIZED] eller [ANONYMIZED] och jag bor i [ANONYMIZED] [ANONYMIZED] på [ANONYMIZED] 24


In [25]:
nlp_config = {
    "nlp_engine_name": "spacy",
    "models": [
        {"lang_code": "sv", "model_name": "sv_core_news_md"},
         ],
}

anonymizer = PresidioReversibleAnonymizer(
    analyzed_fields=["PERSON"],
    languages_config=nlp_config,
)

print(
    anonymizer.anonymize("Hej jag heter Felix, du kan ringa mig på 076-1234567 eller skicka ett mail på test@gmail.com", language="sv"))

SyntaxError: positional argument follows keyword argument (1141381121.py, line 11)

In [14]:
load_dotenv()
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
if OPENAI_API_KEY is None:
    raise ValueError("OPENAI_API_KEY does not exist, add it to env")

In [15]:

# Create the OpenAI client
llm = OpenAI(openai_api_key=OPENAI_API_KEY)

# Define your prompt template
template = """The following is a conversation with an AI assistant. The assistant is helpful, creative, clever, and very friendly.

User: {user_prompt}

AI Assistant: """
prompt = PromptTemplate.from_template(template)

# Initialize the LLMChain with the prompt and the OpenAI LLM client
llm_chain = LLMChain(prompt=prompt, llm=llm)

# Enter the data input
user_prompt = "Hello"

# Run the user prompt through the chain
response = llm_chain.run(user_prompt=user_prompt)
print(response)

Hi there! How can I assist you today?
